In [1]:
# Install required packages
%pip install pandas numpy matplotlib seaborn --quiet

# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)


Note: you may need to restart the kernel to use updated packages.


In [1]:
# Define project goals
goals = """
- Analyze flight delays, cancellations, and trends
- Provide insights at airline and airport level
- Identify causes of delays and seasonal patterns
"""

# Define KPIs
kpis = """
- Average delay minutes (overall, per airline, per airport)
- Cancellation rate (%)
- On-time performance (% flights with delay < 15 min)
- Seasonal delay trends
- Route-level congestion and delays
"""

print("✅ Goals Defined:\n", goals)
print("✅ KPIs Defined:\n", kpis)


✅ Goals Defined:
 
- Analyze flight delays, cancellations, and trends
- Provide insights at airline and airport level
- Identify causes of delays and seasonal patterns

✅ KPIs Defined:
 
- Average delay minutes (overall, per airline, per airport)
- Cancellation rate (%)
- On-time performance (% flights with delay < 15 min)
- Seasonal delay trends
- Route-level congestion and delays



In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)


In [3]:
# %%
# Update path to where your dataset is stored locally
file_path = "DV AirFly Dataset.csv"

# Load dataset
df = pd.read_csv(file_path)

print("✅ Dataset Loaded Successfully!")
print("Shape of dataset:", df.shape)
df.head()


✅ Dataset Loaded Successfully!
Shape of dataset: (3000000, 32)


,FL_DATE,AIRLINE,AIRLINE_DOT,AIRLINE_CODE,DOT_CODE,FL_NUMBER,ORIGIN,ORIGIN_CITY,DEST,DEST_CITY,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,DELAY_DUE_CARRIER,DELAY_DUE_WEATHER,DELAY_DUE_NAS,DELAY_DUE_SECURITY,DELAY_DUE_LATE_AIRCRAFT
0,2019-01-09,United Air Lines Inc.,United Air Lines Inc.: UA,UA,19977,1562,FLL,"Fort Lauderdale, FL",EWR,"Newark, NJ",1155,1151.00,-4.00,19.00,1210.00,1443.00,4.00,1501,1447.00,-14.00,0.00,NaN,0.00,186.00,176.00,153.00,1065.00,NaN,NaN,NaN,NaN,NaN
1,2022-11-19,Delta Air Lines Inc.,Delta Air Lines Inc.: DL,DL,19790,1149,MSP,"Minneapolis, MN",SEA,"Seattle, WA",2120,2114.00,-6.00,9.00,2123.00,2232.00,38.00,2315,2310.00,-5.00,0.00,NaN,0.00,235.00,236.00,189.00,1399.00,NaN,NaN,NaN,NaN,NaN
2,2022-07-22,United Air Lines Inc.,United Air Lines Inc.: UA,UA,19977,459,DEN,"Denver, CO",MSP,"Minneapolis, MN",954,1000.00,6.00,20.00,1020.00,1247.00,5.00,1252,1252.00,0.00,0.00,NaN,0.00,118.00,112.00,87.00,680.00,NaN,NaN,NaN,NaN,NaN
3,2023-03-06,Delta Air Lines Inc.,Delta Air Lines Inc.: DL,DL,19790,2295,MSP,"Minneapolis, MN",SFO,"San Francisco, CA",1609,1608.00,-1.00,27.00,1635.00,1844.00,9.00,1829,1853.00,24.00,0.00,NaN,0.00,260.00,285.00,249.00,1589.00,0.00,0.00,24.00,0.00,0.00
4,2020-02-23,Spirit Air Lines,Spirit Air Lines: NK,NK,20416,407,MCO,"Orlando, FL",DFW,"Dallas/Fort Worth, TX",1840,1838.00,-2.00,15.00,1853.00,2026.00,14.00,2041,2040.00,-1.00,0.00,NaN,0.00,181.00,182.00,153.00,985.00,NaN,NaN,NaN,NaN,NaN


In [4]:
# %%
print("\n--- Schema Info ---")
df.info()

print("\n--- Null Value Summary ---")
print(df.isna().sum())

print("\n--- Basic Statistics ---")
print(df.describe(include='all').transpose())

# Memory usage before optimization
print("\nMemory Usage Before Optimization (MB):", df.memory_usage(deep=True).sum() / 1024**2)



--- Schema Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000000 entries, 0 to 2999999
Data columns (total 32 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   FL_DATE                  object 
 1   AIRLINE                  object 
 2   AIRLINE_DOT              object 
 3   AIRLINE_CODE             object 
 4   DOT_CODE                 int64  
 5   FL_NUMBER                int64  
 6   ORIGIN                   object 
 7   ORIGIN_CITY              object 
 8   DEST                     object 
 9   DEST_CITY                object 
 10  CRS_DEP_TIME             int64  
 11  DEP_TIME                 float64
 12  DEP_DELAY                float64
 13  TAXI_OUT                 float64
 14  WHEELS_OFF               float64
 15  WHEELS_ON                float64
 16  TAXI_IN                  float64
 17  CRS_ARR_TIME             int64  
 18  ARR_TIME                 float64
 19  ARR_DELAY                float64
 20  CANCELLED                

In [5]:
# %%
# Convert low-cardinality object columns to category
for col in df.select_dtypes(include='object').columns:
    if df[col].nunique() < df.shape[0] * 0.5:
        df[col] = df[col].astype('category')

# Downcast numeric columns
for col in df.select_dtypes(include=['int64', 'float64']).columns:
    df[col] = pd.to_numeric(df[col], downcast='unsigned')

# Memory usage after optimization
print("\nMemory Usage After Optimization (MB):", df.memory_usage(deep=True).sum() / 1024**2)



Memory Usage After Optimization (MB): 440.8757858276367


In [7]:
# %%
# Delay and cancellation columns
delay_cols = ["DEP_DELAY", "ARR_DELAY", "DELAY_DUE_CARRIER", "DELAY_DUE_WEATHER",
              "DELAY_DUE_NAS", "DELAY_DUE_SECURITY", "DELAY_DUE_LATE_AIRCRAFT"]
cancellation_cols = ["CANCELLED", "CANCELLATION_CODE"]

# Handle nulls in delay columns
for col in delay_cols:
    if col in df.columns:
        df[col] = df[col].fillna(0)

# Handle nulls in cancellation columns (safe for categorical dtype)
for col in cancellation_cols:
    if col in df.columns:
        if str(df[col].dtype) == "category":
            df[col] = df[col].cat.add_categories(["NA"]).fillna("NA")
        else:
            df[col] = df[col].fillna("NA")

# Convert flight date to datetime
df['FL_DATE'] = pd.to_datetime(df['FL_DATE'], errors='coerce')

print("✅ Nulls handled and FL_DATE formatted")


✅ Nulls handled and FL_DATE formatted


In [9]:
# %%
# Create derived features
df['Month'] = df['FL_DATE'].dt.month
df['DayOfWeek'] = df['FL_DATE'].dt.dayofweek
df['Hour'] = (df['CRS_DEP_TIME'] // 100).astype(int)  # e.g. 1330 -> 13

# Fix for categorical columns when creating Route
df['Route'] = df['ORIGIN'].astype(str) + "-" + df['DEST'].astype(str)

print("✅ Derived Features Added: Month, DayOfWeek, Hour, Route")
df[['FL_DATE', 'Month', 'DayOfWeek', 'Hour', 'Route']].head()


✅ Derived Features Added: Month, DayOfWeek, Hour, Route


,FL_DATE,Month,DayOfWeek,Hour,Route
0,2019-01-09,1,2,11,FLL-EWR
1,2022-11-19,11,5,21,MSP-SEA
2,2022-07-22,7,4,9,DEN-MSP
3,2023-03-06,3,0,16,MSP-SFO
4,2020-02-23,2,6,18,MCO-DFW


In [10]:
# %%
# Save preprocessed dataset
output_path = "DV_AirFly_Preprocessed.csv"
df.to_csv(output_path, index=False)

print("✅ Preprocessed dataset saved at:", output_path)


✅ Preprocessed dataset saved at: DV_AirFly_Preprocessed.csv


In [11]:
# %%
# Preprocessing Summary
preprocessing_summary = {
    "Null Handling": "Delay columns filled with 0, Cancellation columns filled with 'NA'",
    "Derived Features": ["Month", "DayOfWeek", "Hour", "Route"],
    "Datetime Formatting": "FL_DATE converted to datetime",
    "Memory Optimization": "Downcasting numeric, converting categorical",
    "Saved File": output_path
}

print("\n📌 Preprocessing Summary:\n", preprocessing_summary)

# Feature Dictionary
feature_dict = {
    "FL_DATE": "Flight date",
    "AIRLINE": "Airline code",
    "FL_NUMBER": "Flight number",
    "ORIGIN": "Origin airport code",
    "DEST": "Destination airport code",
    "CRS_DEP_TIME": "Scheduled departure time (HHMM)",
    "DEP_TIME": "Actual departure time (HHMM)",
    "DEP_DELAY": "Departure delay in minutes",
    "ARR_DELAY": "Arrival delay in minutes",
    "CANCELLED": "Cancellation flag (1=Yes, 0=No)",
    "CANCELLATION_CODE": "Reason for cancellation",
    "DIVERTED": "Diversion flag",
    "DISTANCE": "Flight distance in miles",
    "Month": "Derived: Month of flight",
    "DayOfWeek": "Derived: Day of week (0=Mon)",
    "Hour": "Derived: Scheduled departure hour",
    "Route": "Derived: ORIGIN-DEST pair"
}

print("\n📌 Feature Dictionary:")
for k, v in feature_dict.items():
    print(f"{k}: {v}")



📌 Preprocessing Summary:
 {'Null Handling': "Delay columns filled with 0, Cancellation columns filled with 'NA'", 'Derived Features': ['Month', 'DayOfWeek', 'Hour', 'Route'], 'Datetime Formatting': 'FL_DATE converted to datetime', 'Memory Optimization': 'Downcasting numeric, converting categorical', 'Saved File': 'DV_AirFly_Preprocessed.csv'}

📌 Feature Dictionary:
FL_DATE: Flight date
AIRLINE: Airline code
FL_NUMBER: Flight number
ORIGIN: Origin airport code
DEST: Destination airport code
CRS_DEP_TIME: Scheduled departure time (HHMM)
DEP_TIME: Actual departure time (HHMM)
DEP_DELAY: Departure delay in minutes
ARR_DELAY: Arrival delay in minutes
CANCELLED: Cancellation flag (1=Yes, 0=No)
CANCELLATION_CODE: Reason for cancellation
DIVERTED: Diversion flag
DISTANCE: Flight distance in miles
Month: Derived: Month of flight
DayOfWeek: Derived: Day of week (0=Mon)
Hour: Derived: Scheduled departure hour
Route: Derived: ORIGIN-DEST pair
